In [136]:
import pandas as pd
import requests
import sqlite3
import time
from tqdm import tqdm
from sqlite3 import Error
from bs4 import BeautifulSoup as bs
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [139]:
def parse_box_score(soup, home, away, date):

    away_advance_html = soup.find_all('table', id = "box-{}-game-advanced".format(away))
    away_basic_html = soup.find_all('table', id = "box-{}-game-basic".format(away))
    home_advance_html = soup.find_all('table', id = "box-{}-game-advanced".format(home))
    home_basic_html = soup.find_all('table', id = "box-{}-game-basic".format(home))


    away_advance = pd.read_html(str(away_advance_html))[0]
    away_basic = pd.read_html(str(away_basic_html))[0]
    home_advance = pd.read_html(str(home_advance_html))[0]
    home_basic = pd.read_html(str(home_basic_html))[0]

    dfs = [home_basic, home_advance, away_basic, away_advance]
    for df in dfs:
        df.columns = df.columns.droplevel(0)
        
    del dfs[1]['MP']
    del dfs[3]['MP']

    df_home = pd.concat([dfs[0].set_index('Starters'),dfs[1].set_index('Starters')], axis=1, join='inner')
    df_away = pd.concat([dfs[2].set_index('Starters'),dfs[3].set_index('Starters')], axis=1, join='inner')

    dfs = [df_home, df_away]
    dfs[0].drop(['Reserves','Team Totals'], axis = 'index', inplace = True)
    dfs[0].reset_index(inplace = True)
    dfs[0]['TEAM'] = home
    dfs[0]['OPP'] = away
    dfs[0]['HCA'] = 1

    dfs[1].drop(['Reserves','Team Totals'], axis = 'index', inplace = True)
    dfs[1].reset_index(inplace = True)
    dfs[1]['TEAM'] = away
    dfs[1]['OPP'] = home
    dfs[1]['HCA'] = 0


    df_return = pd.concat(dfs)
    df_return['DATE'] = date
    df_return['DATE'] = pd.to_datetime(df_return['DATE'])
    return df_return

    
#Get all the available months and their urls
def get_months(year_soup, year):
    month_links = []
    for link in year_soup.find_all('a'):
        if "/leagues/NBA_{}_games-".format(year) in link['href']:
            month_links.append(link['href'])
            
    return month_links

#Get all the links to the boxscores for a given month
def get_box_scores(month):
    url = "https://www.basketball-reference.com" + month
    html_text = requests.get(url).text
    month_page_soup = bs(html_text)

    box_scores_links = []
    for link in month_page_soup.find_all('a'):
        if "/boxscores/2" in link['href']:
            box_scores_links.append(link['href'])

    return box_scores_links

#Get the box score html 
def get_bs_soup(game):
    time.sleep(1)
    date = game[11:19]
    url = "https://www.basketball-reference.com" + game
    html_text = requests.get(url).text
    game_page_soup = bs(html_text)

    #Get home and away team
    a_tags = game_page_soup.find_all('a')
    href_tags = []
    for tag in a_tags:
        try:
            if "/teams/" in tag['href']:
                href_tags.append(tag['href'])
        except:
            pass

    href_tags = href_tags[1:3]
    home = href_tags[1][7:10]
    away = href_tags[0][7:10]
    

    return game_page_soup, home, away, date

In [142]:
def create_conn(db_file):
    
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
        
    except Error as e:
        print(e)
        
    return conn

def main(year):
    #Load the Year HTML page
    url = "https://www.basketball-reference.com/leagues/NBA_{}_games.html".format(year)
    html_text = requests.get(url).text
    main_page_soup = bs(html_text)

    month_links = get_months(main_page_soup, year)
    month_links = [month_links[0]]
    for month in (month_links):
        print(month)
        box_scores_links = get_box_scores(month)

        df_list = []
        
        for game in tqdm(box_scores_links):
            
            game_page_soup, home, away, date = get_bs_soup(game)

            df = parse_box_score(game_page_soup, home, away, date)

            df_list.append(df)
        
        df_month = pd.concat(df_list)
        df_month.reset_index(drop=True, inplace=True)
        
        try:
            del df_month['Unnamed: 16_level_1']
        except:
            pass
        
        df_month.to_sql("MAIN_{}".format(year), conn, if_exists = 'replace')

#December has been aded    
year = "2021"
global conn
conn = create_conn(r"C:\Users\bktor\Desktop\Python\DF\FANTASY.DB")

main(year)

conn.close()

2.6.0
/leagues/NBA_2021_games-december.html


100%|████████████████████████████████████████████████████████████████████████████████| 14/14 [00:19<00:00,  1.43s/it]


In [220]:
SQL = """
SELECT * FROM MAIN
UNION
SELECT * FROM MAIN_2018
UNION
SELECT * FROM MAIN_2019
UNION 
SELECT * FROM MAIN_2020
UNION 
SELECT * FROM MAIN_2021
"""
df = pd.read_sql(SQL, con = conn)
df

,index,Starters,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,TS%,eFG%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM,TEAM,OPP,HCA,DATE
0,0,Al Horford,29:57,4,7,.571,0,1,.000,1,1,1.000,0,4,4,2,0,4,3,1,9,+9,.605,.571,.143,.143,0.0,13.1,6.3,9.0,0.0,10.5,28.7,14.3,90,80,2.5,BOS,PHI,1,2018-10-16 00:00:00
1,0,Anthony Davis,39:33,15,20,.750,1,1,1.000,1,1,1.000,8,6,14,1,1,0,3,4,32,+10,.783,.775,.050,.050,20.7,21.4,21.0,3.9,1.3,0.0,12.8,25.1,143,127,7.1,LAL,MIA,1,2020-10-02 00:00:00
2,0,Blake Griffin,40:02,12,15,.800,4,7,.571,8,9,.889,2,6,8,5,1,0,2,5,36,+22,.949,.933,.467,.600,8.0,14.4,12.0,25.4,1.3,0.0,9.5,24.9,173,110,15.9,LAC,LAL,1,2017-04-01 00:00:00
3,0,Bojan Bogdanović,37:05,7,16,.438,1,3,.333,4,4,1.000,1,6,7,6,3,0,1,3,19,+5,.535,.469,.188,.250,4.0,15.2,10.9,23.6,4.1,0.0,5.3,23.4,121,104,2.8,IND,DET,1,2019-04-01 00:00:00
4,0,Bradley Beal,30:17,8,20,.400,2,8,.250,9,10,.900,1,3,4,5,0,0,2,2,27,-20,.553,.450,.400,.500,3.2,9.9,6.5,39.0,0.0,0.0,7.6,36.8,113,124,4.7,WAS,ORL,1,2020-01-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128655,6156,Sam Dekker,18:40,3,6,.500,2,3,.667,0,0,None,2,3,5,1,0,0,0,0,8,+6,.667,.667,.500,.000,10.7,17.9,14.1,9.8,0.0,0.0,0.0,13.5,156,119,7.9,HOU,GSW,0,2017-03-31 00:00:00
128656,6157,Kyle Wiltjer,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,HOU,GSW,0,2017-03-31 00:00:00
128657,6158,Chinanu Onuaku,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,HOU,GSW,0,2017-03-31 00:00:00
128658,6159,Nenê Hilário,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,HOU,GSW,0,2017-03-31 00:00:00


In [223]:
def get_def_stats(df, _team, dates):
    
    main_opp_list = []
    for date in dates:
        df_op = df[df['OPP'] == _team]
        df_op = df_op[df_op['DATE'] == date]
        df_op = df_op[df_op['MP'] != 'Did Not Play']
        df_op = df_op[df_op['MP'] != 'Not With Team']
        df_op = df_op[df_op['MP'] != 'Player Suspended']
        df_op = df_op[df_op['MP'] != 'Did Not Dress']
        columns = list(df_op.columns.values[4:37])

        df_op[columns] = df_op[columns].apply(pd.to_numeric)

        PTS_A = df_op['PTS'].sum()
        REB_A = df_op['TRB'].sum()
        TOV_A = df_op['TOV'].sum()
        ORT_A = df_op['ORtg'].sum() / len(df_op[df_op['ORtg'] > 0])


        opp_list = [PTS_A, REB_A, TOV_A, ORT_A, date]
        main_opp_list.append(opp_list)

    _df = pd.DataFrame(main_opp_list)
    _df.set_index(4, inplace = True)
    _df.columns = ['PTS_DEF', 'TRB_DEF', 'TOV_DEF', 'ORTG_DEF']
    _df.sort_index(inplace = True)


    _21 = _df.index.to_series().between('2020-10-20', '2021-07-01')
    seasons = [_21]

    df_list = []
    for season in seasons:

        df_mid = _df[season].expanding().mean().shift(1)
        df_list.append(df_mid)

    df_opp_final = pd.concat(df_list)
    df_opp_final['TEAM'] = _team
        
    return df_opp_final

teams = df['TEAM'].unique()
team_dict = {}
for team in teams:
    df_mid = df[df['TEAM'] == team]
    dates = df_mid['DATE'].unique()
    team_dict[team] = dates
teams = list(team_dict.keys())

opp_list = []
for _team in tqdm(teams):
    _dates = team_dict[_team]
    opp_df = get_def_stats(df, _team, _dates)
    opp_list.append(opp_df)


100%|████████████████████████████████████████████████████████████████████████████████| 30/30 [14:04<00:00, 28.14s/it]


In [224]:
opp_df = pd.concat(opp_list)
opp_df.reset_index(inplace = True)
opp_df.columns = ['DATE_DEF', 'PTS_DEF', 'TRB_DEF', 'TOV_DEF','ORTG_DEF','TEAM']
opp_df.to_sql("OPP_DEF", con = conn, if_exists = 'append')

conn = create_conn(r"C:\Users\bktor\Desktop\Python\DF\FANTASY.DB")
SQL_op = """

SELECT * FROM OPP_DEF
"""
df_op = pd.read_sql(SQL_op, con = conn)
df_op.sort_values('DATE_DEF')

2.6.0


,index,DATE_DEF,PTS_DEF,TRB_DEF,TOV_DEF,ORTG_DEF,TEAM
4473,4473,2016-10-25 00:00:00,NaN,NaN,NaN,NaN,CLE
6788,6788,2016-10-25 00:00:00,NaN,NaN,NaN,NaN,SAS
5459,5459,2016-10-25 00:00:00,NaN,NaN,NaN,NaN,NYK
7776,7776,2016-10-25 00:00:00,NaN,NaN,NaN,NaN,POR
2111,2111,2016-10-25 00:00:00,NaN,NaN,NaN,NaN,GSW
...,...,...,...,...,...,...,...
10152,0,2020-12-23 00:00:00,NaN,NaN,NaN,NaN,BOS
10157,5,2020-12-23 00:00:00,NaN,NaN,NaN,NaN,TOR
10156,4,2020-12-23 00:00:00,NaN,NaN,NaN,NaN,WAS
10166,14,2020-12-23 00:00:00,NaN,NaN,NaN,NaN,ATL


In [228]:
df[df['TEAM'] == 'BOS'].sort_values('DATE')

,index,Starters,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,TS%,eFG%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM,TEAM,OPP,HCA,DATE
6188,168,James Young,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,BOS,BRK,1,2016-10-26 00:00:00
6153,167,Jordan Mickey,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,BOS,BRK,1,2016-10-26 00:00:00
6114,166,Demetrius Jackson,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,BOS,BRK,1,2016-10-26 00:00:00
6083,165,Gerald Green,14:45,4,7,.571,1,2,.500,0,0,None,1,4,5,2,0,0,1,1,9,-9,.643,.643,.286,.000,7.9,26.0,17.9,18.6,0.0,0.0,12.5,22.5,121,117,-1.1,BOS,BRK,1,2016-10-26 00:00:00
6026,163,Jonas Jerebko,18:49,1,8,.125,0,5,.000,2,2,1.000,0,1,1,3,4,0,1,1,4,+2,.225,.125,.625,.250,0.0,5.1,2.8,16.8,9.9,0.0,10.1,21.7,68,102,-7.3,BOS,BRK,1,2016-10-26 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7696,212,Tremont Waters,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,BOS,MIL,1,2020-12-23 00:00:00
7462,205,Semi Ojeleye,18:59,2,2,1.000,1,1,1.000,0,1,.000,1,3,4,1,0,0,0,1,5,+20,1.025,1.250,.500,.500,5.0,20.0,11.4,5.9,0.0,0.0,0.0,5.5,203,125,None,BOS,MIL,1,2020-12-23 00:00:00
7339,202,Daniel Theis,24:40,4,8,.500,2,6,.333,0,0,None,0,3,3,3,0,0,0,1,10,+7,.625,.625,.750,.000,0.0,15.4,6.6,14.5,0.0,0.0,0.0,13.9,138,126,None,BOS,MIL,1,2020-12-23 00:00:00
7396,203,Tristan Thompson,22:24,6,10,.600,0,0,None,0,0,None,4,4,8,1,0,0,0,3,12,-6,.600,.600,.000,.000,16.8,22.6,19.3,6.1,0.0,0.0,0.0,19.2,140,125,None,BOS,MIL,1,2020-12-23 00:00:00


In [229]:
teams = df['OPP'].unique()

df_return_list = []
for team in tqdm(teams):
    df_mid = df[df['OPP'] == team]
    df_mid_def = df_op[df_op['TEAM'] == team]

    df_vals = []
    for index, row in df_mid.iterrows():
        date = row['DATE']
        max_df = df_mid_def[df_mid_def['DATE_DEF'] < date]
        try:
            df_vals.append(max_df.tail(1).values[0])
        except:
            df_vals.append([None, None, None, None, None, None, None])

    df_def = pd.DataFrame(df_vals)
    df_def.columns = ['INDEX_DEF', 'DATE_DEF', 'PTS_DEF', 'TRB_DEF','TOV_DEF','ORTG_DEF', 'TEAM_DEF']
    df_return = pd.concat([df_mid.reset_index(drop=True), df_def.reset_index(drop=True)], axis = 1)
    df_return_list.append(df_return)
    
df_final = pd.concat(df_return_list)
df_final.drop(df_final.columns[[0]], axis=1, inplace=True)
df_final

100%|████████████████████████████████████████████████████████████████████████████████| 30/30 [02:50<00:00,  5.69s/it]


,Starters,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,TS%,eFG%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM,TEAM,OPP,HCA,DATE,INDEX_DEF,DATE_DEF,PTS_DEF,TRB_DEF,TOV_DEF,ORTG_DEF,TEAM_DEF
0,Al Horford,29:57,4,7,.571,0,1,.000,1,1,1.000,0,4,4,2,0,4,3,1,9,+9,.605,.571,.143,.143,0.0,13.1,6.3,9.0,0.0,10.5,28.7,14.3,90,80,2.5,BOS,PHI,1,2018-10-16 00:00:00,5944.0,2018-05-09 00:00:00,105.164835,42.109890,13.692308,106.174237,PHI
1,Michael Kidd-Gilchrist,32:16,6,13,.462,0,0,None,4,4,1.000,1,4,5,0,2,0,0,2,16,-21,.542,.462,.000,.308,3.2,15.7,8.9,0.0,2.9,0.0,0.0,19.4,116,116,0.0,CHO,PHI,1,2018-04-01 00:00:00,5927.0,2018-03-30 00:00:00,105.459459,42.108108,13.513514,106.473332,PHI
2,Kemba Walker,25:53,2,7,.286,1,5,.200,5,5,1.000,0,1,1,4,0,1,4,0,10,-12,.543,.357,.714,.714,0.0,4.9,2.2,25.3,0.0,3.0,30.3,21.7,89,121,-5.8,CHO,PHI,1,2018-04-01 00:00:00,5927.0,2018-03-30 00:00:00,105.459459,42.108108,13.513514,106.473332,PHI
3,Kyrie Irving,29:00,2,14,.143,1,8,.125,2,2,1.000,0,4,4,7,0,0,3,1,7,+9,.235,.179,.571,.143,0.0,13.5,6.5,29.9,0.0,0.0,16.8,25.3,63,89,-9.0,BOS,PHI,1,2018-10-16 00:00:00,5944.0,2018-05-09 00:00:00,105.164835,42.109890,13.692308,106.174237,PHI
4,Dwight Howard,25:52,4,7,.571,0,0,None,2,5,.400,1,9,10,0,1,1,1,2,10,-13,.543,.571,.000,.714,4.0,44.0,22.1,0.0,1.8,3.0,9.8,16.8,101,111,-1.6,CHO,PHI,1,2018-04-01 00:00:00,5927.0,2018-03-30 00:00:00,105.459459,42.108108,13.513514,106.473332,PHI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4030,Joe Johnson,17:18,1,4,.250,0,1,.000,0,0,None,0,2,2,0,0,0,2,3,2,+5,.250,.250,.250,.000,0.0,11.3,6.4,0.0,0.0,0.0,33.3,17.2,33,100,-17.6,UTA,PHO,1,2016-12-31 00:00:00,9865.0,2016-12-29 00:00:00,114.312500,43.968750,15.593750,110.545316,PHO
4031,Raul Neto,14:02,3,6,.500,0,2,.000,2,2,1.000,0,1,1,1,1,0,0,2,8,+7,.581,.500,.333,.333,0.0,7.0,3.9,13.8,3.8,0.0,0.0,24.3,124,93,5.5,UTA,PHO,1,2016-12-31 00:00:00,9865.0,2016-12-29 00:00:00,114.312500,43.968750,15.593750,110.545316,PHO
4032,Boris Diaw,14:01,0,2,.000,0,0,None,0,0,None,0,2,2,1,0,0,0,0,0,-5,.000,.000,.000,.000,0.0,14.0,7.9,9.8,0.0,0.0,0.0,7.1,30,99,-7.2,UTA,PHO,1,2016-12-31 00:00:00,9865.0,2016-12-29 00:00:00,114.312500,43.968750,15.593750,110.545316,PHO
4033,Jeff Withey,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,UTA,PHO,1,2016-12-31 00:00:00,9865.0,2016-12-29 00:00:00,114.312500,43.968750,15.593750,110.545316,PHO


In [233]:
df_final.to_sql("MAIN_FINAL", con = conn, if_exists = 'replace')
df_op.to_sql("MAIN_DEF", con = conn, if_exists = 'replace')

In [232]:
df_op

,index,DATE_DEF,PTS_DEF,TRB_DEF,TOV_DEF,ORTG_DEF,TEAM
0,0,2016-10-26 00:00:00,NaN,NaN,NaN,NaN,BOS
1,1,2016-10-27 00:00:00,117.000000,44.000000,16.00,124.250000,BOS
2,2,2016-10-29 00:00:00,111.000000,49.500000,14.00,114.375000,BOS
3,3,2016-11-02 00:00:00,106.666667,45.666667,12.00,110.616667,BOS
4,4,2016-11-03 00:00:00,105.000000,46.500000,13.75,110.337500,BOS
...,...,...,...,...,...,...,...
10175,23,2020-12-23 00:00:00,NaN,NaN,NaN,NaN,POR
10176,24,2020-12-23 00:00:00,NaN,NaN,NaN,NaN,UTA
10177,25,2020-12-23 00:00:00,NaN,NaN,NaN,NaN,DAL
10178,26,2020-12-23 00:00:00,NaN,NaN,NaN,NaN,SAC
